In [ ]:
import numpy as np
from numpy import random
import json

import string
alphabet_str = string.ascii_lowercase
print(alphabet_str)

# As for the encoding schema, 
# let's start with a general fixed letter to fixed letter permutation first.

# Choose one of the 26! letter permutations randomly. 
# With a fixed seed parameter the same one can be chosen repeatedly.
def construct_random_letter_permutation(seed=None):
    if seed != None:
        random.seed(seed)
    letter_indices = np.array(list(range(0,len(alphabet_str))))
    #print(letter_indices)
    letter_indices_permutated = letter_indices.copy()
    letter_indices_permutated = random.permutation(letter_indices_permutated)
    #print(letter_indices_permutated)    
    letter_permutation_dict = {}
    for letter_idx_in in letter_indices:
        letter_idx_out = letter_indices_permutated[letter_idx_in]
        letter_permutation_dict[alphabet_str[letter_idx_in]] = alphabet_str[letter_idx_out] 
    return letter_permutation_dict

# enoode a message with a letter permutation
def encode(msg, letter_permutation):
    enc_msg = ''
    for i in range(len(msg)):
        enc_msg += letter_permutation[msg[i]]
    return enc_msg

In [ ]:
plaintext_msg = 'Let us try to decrypt with a quantum computer.'
# Remove spaces and dots, and restrict to lowercase characters only, so we have 26 symbols.
plaintext_msg = plaintext_msg.replace(' ', '').replace('.', '').lower()

print('plaintext message = {:s}'.format(plaintext_msg))
letter_permutation_dict = construct_random_letter_permutation(3)
print('letter permutation dictionary =')
print(json.dumps(letter_permutation_dict, indent=2))
encoded_msg = encode(plaintext_msg, letter_permutation_dict)
print('encoded message = {:s}'.format(encoded_msg))

In [ ]:
inverse_letter_permutation_dict = {v: k for k, v in letter_permutation_dict.items()}
print('inverse letter permutation dictionary =')
print(json.dumps(inverse_letter_permutation_dict, indent=2))
recovered_msg = encoded_msg = encode(encoded_msg, inverse_letter_permutation_dict)
print('recovered message = {:s}'.format(recovered_msg))

In [ ]:
"""  From view-source:http://norvig.com/mayzner.html, we get:
E 445.2 B  12.49% <img src="o.jpg" height=12 width=374> E
T 330.5 B   9.28% <img src="o.jpg" height=12 width=278> T
A 286.5 B   8.04% <img src="o.jpg" height=12 width=241> A
O 272.3 B   7.64% <img src="o.jpg" height=12 width=229> O
I 269.7 B   7.57% <img src="o.jpg" height=12 width=227> I
N 257.8 B   7.23% <img src="o.jpg" height=12 width=217> N
S 232.1 B   6.51% <img src="o.jpg" height=12 width=195> S
R 223.8 B   6.28% <img src="o.jpg" height=12 width=188> R
H 180.1 B   5.05% <img src="o.jpg" height=12 width=151> H
L 145.0 B   4.07% <img src="o.jpg" height=12 width=122> L
D 136.0 B   3.82% <img src="o.jpg" height=12 width=114> D
C 119.2 B   3.34% <img src="o.jpg" height=12 width=100> C
U  97.3 B   2.73% <img src="o.jpg" height=12 width=81> U
M  89.5 B   2.51% <img src="o.jpg" height=12 width=75> M
F  85.6 B   2.40% <img src="o.jpg" height=12 width=72> F
P  76.1 B   2.14% <img src="o.jpg" height=12 width=64> P
G  66.6 B   1.87% <img src="o.jpg" height=12 width=56> G
W  59.7 B   1.68% <img src="o.jpg" height=12 width=50> W
Y  59.3 B   1.66% <img src="o.jpg" height=12 width=49> Y
B  52.9 B   1.48% <img src="o.jpg" height=12 width=44> B
V  37.5 B   1.05% <img src="o.jpg" height=12 width=31> V
K  19.3 B   0.54% <img src="o.jpg" height=12 width=16> K
X   8.4 B   0.23% <img src="o.jpg" height=12 width=7> X
J   5.7 B   0.16% <img src="o.jpg" height=12 width=4> J
Q   4.3 B   0.12% <img src="o.jpg" height=12 width=3> Q
Z   3.2 B   0.09% <img src="o.jpg" height=12 width=2> Z
"""